# Variational Quantum State Diagonalization 

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## Overview

- In this tutorial, we will train a quantum neural network (QNN) through Paddle Quantum to complete the diagonalization of quantum states.

- First, import the following packages.

In [1]:
import numpy
from numpy import diag
from numpy import pi as PI
import scipy
import scipy.stats
import paddle
from paddle import matmul, trace
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import dagger

## Background
The Variational Quantum State Diagonalization [1-3] aims to output the eigen-spectrum (eigenvalues) of a quantum state. Solving the eigenvalues ​​of quantum states has many applications in quantum computation, such as calculating fidelity and Von Neumann entropy.

- Quantum state is usually a mixed state which can be expressed as follows: 

$$
\rho_{\text{mixed}} = \sum_i P_i |\psi_i\rangle\langle\psi_i|. \tag{1}
$$

- As an example, we consider a mixed 2-qubit quantum state with eigen-spectrum $[0.5, 0.3, 0.1, 0.1]$.

In [2]:
# Fixed random seed
scipy.random.seed(13) 
V = scipy.stats.unitary_group.rvs(4)  # Randomly generate a unitary matrix
D = diag([0.5, 0.3, 0.1, 0.1])        # Input the spectrum of the target state rho
V_H = V.conj().T                      # Conjugate transpose operation
rho = V @ D @ V_H                     # Generate rho by inverse spectral decomposition
print(numpy.around(rho, 4))           # Print quantum state rho

[[ 0.2569+0.j     -0.012 +0.0435j -0.0492-0.0055j -0.0548+0.0682j]
 [-0.012 -0.0435j  0.2959-0.j      0.1061-0.0713j -0.0392-0.0971j]
 [-0.0492+0.0055j  0.1061+0.0713j  0.2145-0.j      0.0294-0.1132j]
 [-0.0548-0.0682j -0.0392+0.0971j  0.0294+0.1132j  0.2327+0.j    ]]


## Building a quantum neural network

- In this case, we will learn the eigen-spectrum of quantum state $\rho$ defined above by training a QNN (also known as the parameterized quantum circuit). Here, we provide a predefined 2-qubit quantum circuit.

- One can randomly initialize the QNN parameters ${\bf{\vec{\theta }}}$ containing 15 parameters.

In [3]:
N = 2           # The width of the quantum neural network
SEED = 14       # Fixed random seed
THETA_SIZE = 15 # The number of parameters in the quantum neural network

def U_theta(theta, N):
    """
    Quantum Neural Network
    """
    # Initialize the quantum neural network according to the number of qubits/network width
    cir = UAnsatz(N)
    # Call the built-in quantum neural network template
    cir.universal_2_qubit_gate(theta, [0, 1])
    # Return the unitary matrix U simulated by the quantum neural network
    return cir.U

## Training model and loss function

- After setting up the quantum state and the QNN architecture, we will further define the parameters to be trained, loss function, and optimization methods. 
- The quantum state obtained by acting the quantum neural network $U(\theta)$ on $\rho$  is denoted by $\tilde\rho$, and we set the loss function to be the inner product of the quantum state $\sigma$ and $\tilde\rho$ where

$$
\sigma=0.1 |00\rangle\langle 00| + 0.2 |01\rangle \langle 01| + 0.3 |10\rangle \langle10| + 0.4 |11 \rangle\langle 11|, \tag{2}
$$

- In specific, the loss function is defined as the state overlap 

$$
\mathcal{L}(\boldsymbol{\theta}) = \text{Tr}(\tilde\rho\sigma). \tag{3}
$$

In [4]:
# Enter the quantum state sigma 
sigma = diag([0.1, 0.2, 0.3, 0.4]).astype('complex128')

class Net(paddle.nn.Layer):
    """
    Construct the model net
    """

    def __init__(self, shape, rho, sigma, dtype='float64'):
        super(Net, self).__init__()
        
        
        
        # Convert Numpy array to Tensor supported in Paddle 
        self.rho = paddle.to_tensor(rho)
        self.sigma = paddle.to_tensor(sigma)
        
        # Initialize the theta parameter list and fill the initial value with the uniform distribution of [0, 2*pi]
        self.theta = self.create_parameter(shape=shape, 
                                           default_initializer=paddle.nn.initializer.Uniform(low=0.0, high=2*PI),
                                           dtype=dtype, is_bias=False)

    # Define loss function and forward propagation mechanism
    def forward(self, N):
        
        # Apply quantum neural network
        U = U_theta(self.theta, N)

        # rho_tilde is the quantum state U*rho*U^dagger 
        rho_tilde = matmul(matmul(U, self.rho), dagger(U))

        # Calculate the loss function
        loss = trace(matmul(self.sigma, rho_tilde))

        return paddle.real(loss), rho_tilde

## Hyper-parameters

Before training the quantum neural network, we also need to set up several hyper-parameters, mainly the learning rate LR and the number of iterations ITR. Here we set the learning rate to be LR = 0.1 and the number of iterations to ITR = 50. One can adjust these hyper-parameters accordingly and check how they influence the training performance.

In [5]:
ITR = 50 # Set the total number of iterations of training
LR = 0.1 # Set the learning rate

## Training process

- After setting all the parameters of SSVQE model, we need to convert all the data into Tensor in the PaddlePaddle, and then train the quantum neural network.
- We used Adam Optimizer in training, and one can also call other optimizers provided in Paddle.

In [6]:
paddle.seed(SEED)

# Determine the parameter dimension of the network
net = Net(shape=[THETA_SIZE], rho=rho, sigma=sigma)

# We use Adam optimizer for better performance
# One can change it to SGD or RMSprop.
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.parameters())

# Optimization loop
for itr in range(ITR):

    # Forward propagation calculates the loss function and returns the estimated energy spectrum
    loss, rho_tilde = net(N)
    rho_tilde_np = rho_tilde.numpy()

    # Back propagation minimizes the loss function
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()

    # Print training results
    if itr% 10 == 0:
        print('iter:', itr,'loss:','%.4f'% loss.numpy()[0])

iter: 0 loss: 0.2494
iter: 10 loss: 0.1958
iter: 20 loss: 0.1843
iter: 30 loss: 0.1816
iter: 40 loss: 0.1805


## Benchmarking

After 50 iterations, we have completed the diagonalization procedure. The next step is to verify the results of spectral decomposition by printing out $\tilde{\rho} = U(\boldsymbol{\theta})\rho U^\dagger(\boldsymbol{\theta})$. One can see the results are very close to what we expect.

In [7]:
print("The estimated spectrum is:", numpy.real(numpy.diag(rho_tilde_np)))
print("The target spectrum is:", numpy.diag(D))

The estimated spectrum is: [0.49938069 0.29916354 0.10103808 0.10041768]
The target spectrum is: [0.5 0.3 0.1 0.1]


_______


## References

[1] Larose, R., Tikku, A., Neel-judy, É. O., Cincio, L. & Coles, P. J. Variational quantum state diagonalization. [npj Quantum Inf. (2019) doi:10.1038/s41534-019-0167-6.](https://www.nature.com/articles/s41534-019-0167-6)

[2] Nakanishi, K. M., Mitarai, K. & Fujii, K. Subspace-search variational quantum eigensolver for excited states. [Phys. Rev. Res. 1, 033062 (2019).](https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.1.033062)

[3] Cerezo, M., Sharma, K., Arrasmith, A. & Coles, P. J. Variational Quantum State Eigensolver. [arXiv:2004.01372 (2020).](https://arxiv.org/pdf/2004.01372.pdf)